In [12]:
# Importing necessary libraries
import pandas as pd
import sqlite3

# Connect to the SQLite database
database_path = 'nba_salary.sqlite'  # Replace with your file path if needed
conn = sqlite3.connect(database_path)

# Function to preview data from a table
def preview_table(connection, table_name, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    return pd.read_sql_query(query, connection)

# Fetch and preview table names
def get_table_names(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql_query(query, connection)

print("Tables in the database:")
print(get_table_names(conn))

# Preview data from the tables
nba_salary_preview = preview_table(conn, "NBA_season1718_salary")
season_stats_preview = preview_table(conn, "Seasons_Stats")


# Perform analysis: Join salary and performance data
query = """
SELECT 
    salary.Player,
    salary.Tm AS Team,
    salary.season17_18 AS Salary,
    stats.PER,
    stats.PTS,
    stats.WS,
    stats.Age,
    stats.Pos
FROM 
    NBA_season1718_salary AS salary
JOIN 
    Seasons_Stats AS stats
ON 
    salary.Player = stats.Player
WHERE 
    stats.Year = 2017
ORDER BY 
    Salary DESC;
"""
analysis_result = pd.read_sql_query(query, conn)

# Display the analysis results
print("Analysis: Salary vs. Performance")
print(analysis_result)

# Close the database connection


Tables in the database:
                    name
0  NBA_season1718_salary
1          Seasons_Stats
Analysis: Salary vs. Performance
             Player Team      Salary   PER     PTS    WS   Age Pos
0     Stephen Curry  GSW  34682550.0  24.6  1999.0  12.6  28.0  PG
1      LeBron James  CLE  33285709.0  27.0  1954.0  12.9  32.0  SF
2      Paul Millsap  DEN  31269231.0  17.8  1246.0   6.4  31.0  PF
3    Gordon Hayward  BOS  29727900.0  22.2  1601.0  10.4  26.0  SF
4     Blake Griffin  DET  29512900.0  22.7  1316.0   7.7  27.0  PF
..              ...  ...         ...   ...     ...   ...   ...  ..
547    Axel Toupane  NOP     25000.0  -9.9     0.0   0.0  24.0  SF
548    Axel Toupane  NOP     25000.0   8.6    11.0   0.1  24.0  SF
549      Beno Udrih  DET     25000.0  16.1   227.0   1.3  34.0  PG
550   Joel Bolomboy  MIL     22248.0  19.7    22.0   0.2  23.0  PF
551    Jarell Eddie  CHI     17224.0   9.7    24.0   0.1  25.0  SF

[552 rows x 8 columns]


In [13]:
print("NBA Salary Table Preview:")
print(nba_salary_preview.head())

NBA Salary Table Preview:
    X1          Player   Tm  season17_18
0  1.0   Stephen Curry  GSW   34682550.0
1  2.0    LeBron James  CLE   33285709.0
2  3.0    Paul Millsap  DEN   31269231.0
3  4.0  Gordon Hayward  BOS   29727900.0
4  5.0   Blake Griffin  DET   29512900.0


In [14]:
print("Season Stats Table Preview:")
print(season_stats_preview.head())

Season Stats Table Preview:
    X1    Year           Player  Pos   Age   Tm     G    GS    MP   PER  ...  \
0  0.0  1950.0  Curly Armstrong  G-F  31.0  FTW  63.0  None  None  None  ...   
1  1.0  1950.0     Cliff Barker   SG  29.0  INO  49.0  None  None  None  ...   
2  2.0  1950.0    Leo Barnhorst   SF  25.0  CHS  67.0  None  None  None  ...   
3  3.0  1950.0       Ed Bartels    F  24.0  TOT  15.0  None  None  None  ...   
4  4.0  1950.0       Ed Bartels    F  24.0  DNN  13.0  None  None  None  ...   

     FT%   ORB   DRB   TRB    AST   STL   BLK   TOV     PF    PTS  
0  0.705  None  None  None  176.0  None  None  None  217.0  458.0  
1  0.708  None  None  None  109.0  None  None  None   99.0  279.0  
2  0.698  None  None  None  140.0  None  None  None  192.0  438.0  
3  0.559  None  None  None   20.0  None  None  None   29.0   63.0  
4  0.548  None  None  None   20.0  None  None  None   27.0   59.0  

[5 rows x 53 columns]


In [15]:
query = """
SELECT COUNT(*) AS total_players FROM NBA_season1718_salary;
"""
output = pd.read_sql(query, conn)
output.head()


,total_players
0,573


In [16]:
query = """
SELECT COUNT(*) AS players_above_20M 
FROM NBA_season1718_salary 
WHERE season17_18 > 20000000;
"""
output = pd.read_sql(query, conn)
output.head()


,players_above_20M
0,40


In [17]:
query = """
SELECT AVG(season17_18) AS average_salary 
FROM NBA_season1718_salary;
"""
output = pd.read_sql(query, conn)
output.head()


,average_salary
0,5.858946e+06


In [18]:
query = """
SELECT Player, Tm AS Team, season17_18 AS Salary 
FROM NBA_season1718_salary 
WHERE season17_18 > (SELECT AVG(season17_18) FROM NBA_season1718_salary)
ORDER BY season17_18 DESC;
"""
output = pd.read_sql(query, conn)
output.head()

,Player,Team,Salary
0,Stephen Curry,GSW,34682550.0
1,LeBron James,CLE,33285709.0
2,Paul Millsap,DEN,31269231.0
3,Gordon Hayward,BOS,29727900.0
4,Blake Griffin,DET,29512900.0


In [19]:
query = """
SELECT Tm AS Team, COUNT(Player) AS total_players 
FROM NBA_season1718_salary 
GROUP BY Tm
ORDER BY total_players DESC;
"""
output = pd.read_sql(query, conn)
output.head()

,Team,total_players
0,ATL,27
1,CHI,24
2,MIL,22
3,LAL,22
4,HOU,22


In [20]:
query = """
SELECT Pos AS Position, AVG(PTS) AS avg_points, AVG(WS) AS avg_win_shares 
FROM Seasons_Stats 
WHERE Year = 2017
GROUP BY Pos
ORDER BY avg_points DESC;
"""
output = pd.read_sql(query, conn)
output.head()


,Position,avg_points,avg_win_shares
0,PG,514.534483,2.255172
1,SG,496.840000,1.780000
2,SF,487.851240,2.336364
3,C,468.964602,3.050442
4,PF,405.689076,1.948739


In [21]:
query = """
SELECT Player, Tm AS Team, PER, PTS, WS 
FROM Seasons_Stats 
WHERE Year = 2017 
ORDER BY PER DESC
LIMIT 10;
"""
output = pd.read_sql(query, conn)
output.head()


,Player,Team,PER,PTS,WS
0,Jarnell Stokes,DEN,31.5,3.0,0.1
1,Demetrius Jackson,BOS,30.8,10.0,0.1
2,Russell Westbrook,OKC,30.6,2558.0,13.1
3,Boban Marjanovic,DET,29.6,191.0,1.7
4,Kevin Durant,GSW,27.6,1555.0,12.0


In [22]:
query = """
SELECT 
    salary.Player,
    salary.Tm AS Team,
    salary.season17_18 AS Salary,
    stats.PER,
    stats.PTS,
    stats.WS,
    stats.Age,
    stats.Pos
FROM 
    NBA_season1718_salary AS salary
JOIN 
    Seasons_Stats AS stats
ON 
    salary.Player = stats.Player
WHERE 
    stats.Year = 2017
ORDER BY 
    Salary DESC
LIMIT 10;
"""
output = pd.read_sql(query, conn)
output.head()


,Player,Team,Salary,PER,PTS,WS,Age,Pos
0,Stephen Curry,GSW,34682550.0,24.6,1999.0,12.6,28.0,PG
1,LeBron James,CLE,33285709.0,27.0,1954.0,12.9,32.0,SF
2,Paul Millsap,DEN,31269231.0,17.8,1246.0,6.4,31.0,PF
3,Gordon Hayward,BOS,29727900.0,22.2,1601.0,10.4,26.0,SF
4,Blake Griffin,DET,29512900.0,22.7,1316.0,7.7,27.0,PF


In [23]:
query = """
SELECT Pos AS Position, AVG(Age) AS average_age 
FROM Seasons_Stats 
WHERE Year = 2017
GROUP BY Pos
ORDER BY average_age ASC;
"""
output = pd.read_sql(query, conn)
output.head()

,Position,average_age
0,PF-C,25.000000
1,PF,26.075630
2,SG,26.248000
3,C,26.265487
4,PG,26.551724


In [24]:
query = """
SELECT Tm AS Team, SUM(season17_18) AS total_salary 
FROM NBA_season1718_salary 
GROUP BY Tm
ORDER BY total_salary DESC
LIMIT 5;
"""
output = pd.read_sql(query, conn)
output.head()


,Team,total_salary
0,GSW,137494845.0
1,CLE,137288549.0
2,OKC,134294056.0
3,MIA,131222624.0
4,WAS,124179842.0


In [25]:
query = """
SELECT 
    salary.Player,
    salary.Tm AS Team,
    salary.season17_18 AS Salary,
    stats.PTS,
    stats.WS
FROM 
    NBA_season1718_salary AS salary
JOIN 
    Seasons_Stats AS stats
ON 
    salary.Player = stats.Player
WHERE 
    stats.Year = 2017
ORDER BY 
    Salary DESC
LIMIT 20;
"""
output = pd.read_sql(query, conn)
output.head()

,Player,Team,Salary,PTS,WS
0,Stephen Curry,GSW,34682550.0,1999.0,12.6
1,LeBron James,CLE,33285709.0,1954.0,12.9
2,Paul Millsap,DEN,31269231.0,1246.0,6.4
3,Gordon Hayward,BOS,29727900.0,1601.0,10.4
4,Blake Griffin,DET,29512900.0,1316.0,7.7


In [26]:
query = """
SELECT 
    ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM Seasons_Stats WHERE Year = 2017)), 2) AS percentage_high_PER
FROM 
    Seasons_Stats
WHERE 
    Year = 2017 AND PER > 20;
"""
output = pd.read_sql(query, conn)
output.head()


,percentage_high_PER
0,9.58


In [27]:
query = """
SELECT Tm AS Team, SUM(WS) AS total_win_shares 
FROM Seasons_Stats 
WHERE Year = 2017
GROUP BY Tm
ORDER BY total_win_shares DESC
LIMIT 5;
"""
output = pd.read_sql(query, conn)
output.head()


,Team,total_win_shares
0,TOT,90.9
1,GSW,67.9
2,SAS,60.4
3,HOU,55.3
4,TOR,54.1


In [28]:
query = """
SELECT Player, Age, PTS, WS, PER 
FROM Seasons_Stats 
WHERE Year = 2017 AND Age BETWEEN 25 AND 30
ORDER BY PTS DESC
LIMIT 10;
"""
output = pd.read_sql(query, conn)
output.head()

,Player,Age,PTS,WS,PER
0,Russell Westbrook,28.0,2558.0,13.1,30.6
1,James Harden,27.0,2356.0,15.0,27.3
2,Isaiah Thomas,27.0,2199.0,12.6,26.5
3,Damian Lillard,26.0,2024.0,10.3,24.1
4,DeMar DeRozan,27.0,2020.0,9.0,24.0


In [29]:
query = """
SELECT stats.Pos AS Position, AVG(salary.season17_18) AS avg_salary 
FROM NBA_season1718_salary AS salary
JOIN Seasons_Stats AS stats
ON salary.Player = stats.Player
WHERE stats.Year = 2017
GROUP BY stats.Pos
ORDER BY avg_salary DESC;
"""
output = pd.read_sql(query, conn)
output.head()

,Position,avg_salary
0,C,8.080497e+06
1,PF,6.601501e+06
2,SF,6.590565e+06
3,SG,5.779589e+06
4,PG,5.567825e+06


In [30]:
query = """
SELECT Player, Tm AS Team, season17_18 AS Salary 
FROM (
    SELECT *, 
           RANK() OVER (PARTITION BY Tm ORDER BY season17_18 DESC) AS rank
    FROM NBA_season1718_salary
) 
WHERE rank <= 3
ORDER BY Team, Salary DESC;
"""
output = pd.read_sql(query, conn)
output.head()


,Player,Team,Salary
0,Kent Bazemore,ATL,16910113.0
1,Dennis Schroder,ATL,15500000.0
2,Miles Plumlee,ATL,12500000.0
3,Gordon Hayward,BOS,29727900.0
4,Al Horford,BOS,27734405.0


In [31]:
conn.close()